In [ ]:
#region imports
from AlgorithmImports import *
#endregion

class BasicTemplateSPXWeeklyIndexOptionsAlgorithm(QCAlgorithm):
    def Initialize(self):
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2020, 12, 31)
        self.SetCash(1000000)

        self.spx = self.AddIndex("SPX", market=Market.USA).Symbol

        # regular option SPX contracts
        self.spxOptions = self.AddIndexOption(self.spx)
        self.spxOptions.SetFilter(lambda u: (u.Strikes(2,5).Expiration(1,5)))

        # weekly option SPX contracts
        spxw = self.AddIndexOption(self.spx, "SPXW")
        #spxw.SetFilter(1, 10, timedelta(1), timedelta(5))
        # set our strike/expiry filter for this option chain
        #spxw.SetFilter(lambda u: (u.Strikes(0, 1)
                                     # single week ahead since there are many SPXW contracts and we want to preserve performance
         #                            .Expiration(0, 7)
          #                           .IncludeWeeklys()))

        self.spxw_option = spxw.Symbol

    def OnData(self, slice):
        if self.Portfolio.Invested:
            return

        for canonicalSymbol, chain in slice.OptionChains.items():
            if not chain.Contracts:
                continue

            # Filter contracts with non-zero volume and only call options
            activecontracts = [contract for contract in chain.Contracts.values() if contract.Volume > 0 and contract.Right == OptionRight.Call]

            # If there are no active contracts, continue to the next chain
            if not activecontracts:
                continue

            # Find the contract with the maximum strike price
            #maxstrikecontract = max(activecontracts, key=lambda x: x.Symbol.ID.StrikePrice)
            sorted_contracts = sorted(activecontracts, key=lambda x: x.Symbol.ID.StrikePrice, reverse=True)
            max_two_strike_contracts = sorted_contracts[:2]

            # Execute a market order to short 5 contracts of the maximum strike option
            self.MarketOrder(max_two_strike_contracts[-1].Symbol, -5)
            #self.MarketOrder(max_two_strike_contracts[1].Symbol, -5)
            #self.Debug(f"Shorted 5 contracts of {maxstrikecontract.Symbol} with strike {maxstrikecontract.StrikePrice}")

def OnOrderEvent(self, orderEvent):
    self.Debug(str(orderEvent))


    def OnOrderEvent(self, orderEvent):
        self.Debug(str(orderEvent))

    def PlotData(self):
        plt.plot(self.strikes, self.spy_prices, marker='o', linestyle='-', color='b')
        plt.xlabel('Option Strike Price')
        plt.ylabel('SPX Price')
        plt.title('SPX Price vs. Option Strike')
        plt.show()
    
    def OnEndOfAlgorithm(self):
        if not self.LiveMode:
            for trade in self.TradeBuilder.ClosedTrades:
                self.Debug(self.trade_to_string(trade))

    def trade_to_string(self, trade):
        profit_loss = 'Profit' if trade.ProfitLoss > 0 else 'Loss'
        return 'Date: {0} Symbol: {1} Profit/Loss: {2} {3}'.format(
            trade.ExitTime,
            trade.Symbol,
            profit_loss,
            trade.ProfitLoss)